In [ ]:
#install dependencies,to install you can uncomment the comment lines below. 
# !pip install layoutparser
# !pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2' 
# !pip install layoutparser[ocr]  
# !sudo apt-get install poppler-utils
# !sudo apt install tesseract-ocr

import layoutparser as lp
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import cv2 
from pdf2image import pdfinfo_from_path,convert_from_path
import numpy as np
import time




def pdf_to_text(pdf, output_file = 'output.txt'):

  '''
  pdf: path to the pdf that you want to extract.
  output_file: output of  the text file
  '''
  start = time.time()

  images = convert_from_path(pdf, fmt='jpeg', poppler_path=None)

  model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config', 
                                extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
  all_text = ''

  for i in range(len(images)):
  
    image = np.array(images[i])
    layout = model.detect(image)

    text_blocks = lp.Layout([b for b in layout if b.type=='Text'])
    figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])
    text_blocks = lp.Layout([b for b in text_blocks \
                  if not any(b.is_in(b_fig) for b_fig in figure_blocks)])

    h, w = image.shape[:2]
    left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)
    left_blocks = text_blocks.filter_by(left_interval, center=True)
    # The b.coordinates[1] corresponds to the y coordinate of the region
    # sort based on that can simulate the top-to-bottom reading order
    right_blocks = lp.Layout([b for b in text_blocks if b not in left_blocks])
    right_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)

    # And finally combine the two lists and add the index
    text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

    ocr_agent = lp.TesseractAgent(languages='eng') 

    for block in text_blocks:
      segment_image = (block
                      .pad(left=5, right=5, top=5, bottom=5)
                      .crop_image(image))
      text = ocr_agent.detect(segment_image)
      block.set(text=text, inplace=True)
      
    samp_text = ''
    for txt in text_blocks.get_texts():
      samp_text = samp_text + txt + '\n' 
    all_text = all_text + samp_text + '_____________________________________________page{}____________________________________'.format(i+1) + '\n'
    all_text = all_text.replace('\f','')

  with open(output_file, "w") as text_file:
    text_file.write("%s" % all_text)

  print('Time spent to extract the txt file in minutes is:',(time.time() - start)/60)


#Used cases
#pdf_to_text('Boskalis_Sustainability_Report_2020.pdf')
#pdf_to_text('Boskalis_Sustainability_Report_2020.pdf', 'final_output.txt')